In [ ]:
NEO4J_URI = "neo4j+s://7d3d4dbd.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "4cQimVVQmSVbF5rh6BA-qjWJcIQkmRnyjKXMoxco0Pw"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def test_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 1")
            return result.single()[0]
    except Exception as e:
        return f"Erreur de connexion : {str(e)}"
    finally:
        driver.close()

print(test_connection())


In [ ]:
def load_books():
    with driver.session() as session:
        query = """
        MATCH (b:Book)
        RETURN b.ISBN AS ISBN, b.title AS title, b.author AS author
        """
        result = session.run(query)
        return [Book(**record) for record in result]

def load_ratings():
    with driver.session() as session:
        query = """
        MATCH (u:User)-[r:RATED]->(b:Book)
        RETURN u["UserId"] AS user_id, b.ISBN AS book_id, toInteger(r.rating) AS rating
        """
        result = session.run(query)
        return pd.DataFrame([record.data() for record in result])


In [ ]:
class Book(BaseModel):
    ISBN: str
    title: str
    description: Optional[str] = ""
    author: Optional[str] = ""

class Recommendation(BaseModel):
    isbn: str
    title: str
    score: float
    author: str